### Purpose of notebook
The purpose of this notebook is to pass the YouTube ASL videos through the MoviNet backbone and then save its output back to S3

In [1]:
# Let's not show unnecessary warnings etc
import warnings
warnings.filterwarnings('ignore')

In [2]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [3]:
# AWS credentials and S3 settings
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
s3_URI = 's3://asl-capstone/'

In [4]:
import s3fs
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

import boto3
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')

In [5]:
# Import all required libraries. Keep adding here as you code
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

tf.keras.backend.clear_session()

2023-11-17 03:17:28.846350: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 03:17:28.846401: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 03:17:28.846421: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 03:17:28.853820: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 03:17:29.759792: W tensorflow/compiler/

In [6]:
# Set working directory
%cd /home/ec2-user/models

/home/ec2-user/models


In [7]:
# Define the model you are going to use
model_version = 'a3'

if model_version=='a3':
  max_frames = 120
  image_dims = (256,256)
elif model_version=='a0':
  max_frames = 50
  image_dims = (172,172)
elif model_version=='a5':
  max_frames = 120
  image_dims = (320,302)

In [8]:
# Get the kinetics-600 action labels
KINETICS_URL = "official/projects/movinet/files/kinetics_600_labels.txt"
with open(KINETICS_URL) as obj:
  labels_600 = [line for line in obj.readlines()]
print("Found %d labels." % len(labels_600))

Found 600 labels.


In [9]:
# Create the backbone using the Movinet model
from official.projects.movinet.modeling import movinet

# Create backbone and model.
backbone = movinet.Movinet(
    model_id=model_version, #change to correspond to model
    causal=False,
    use_external_states=False,
)

2023-11-17 03:17:35.523963: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-17 03:17:35.545787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-17 03:17:35.547773: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
# Get a list of all the numpy files from S3

paginator = s3.get_paginator('list_objects_v2')
video_files = []
for page in paginator.paginate(Bucket = bucket_name, Prefix = 'youtube-asl/1000-samples/numpy_files/'):
    video_files.extend(content['Key'] for content in page.get('Contents',[]) if content['Key'].endswith(('.npy')))
print(len(video_files))

59042


In [11]:
# Add the S3 prefix
video_files = ['s3://asl-capstone/'+x for x in video_files]

In [12]:
# Let's break this into batches
video_files = video_files[:20000]

In [13]:
# Iteration batch
iteration_1 = video_files[:5000]
iteration_2 = video_files[5001:10000]
iteration_3 = video_files[10001:15000]
iteration_4 = video_files[15000:]

In [14]:
movinet_uri = 's3://asl-capstone/youtube-asl/1000-samples/movinet/backbone/'

In [15]:
print(iteration_1[0])

s3://asl-capstone/youtube-asl/1000-samples/numpy_files/--6bmFM9wT4_0.npy


In [16]:
# Initialize the progress bar with the total number of iterations
#pbar = tqdm(total=len(iteration_1))

  0%|                                                                                                                                                                               | 0/5000 [00:00<?, ?it/s]

In [18]:
for vid in tqdm(iteration_1):
    # Generate video embeddings and store to CPU
    filename = os.path.basename(vid)
    with fs.open(vid,"rb") as f:
        vid_file = np.load(f)
    vid_file = np.expand_dims(vid_file, axis=0)
    embeddings = backbone(vid_file)[0]['block4_layer3'].cpu().numpy()
    with fs.open(f"{movinet_uri+filename}","wb") as f:
        np.save(f,embeddings)
    #pbar.update(1)
#pbar.close() #Closing the bar


  0%|                                                                                                                                                                    | 2/5000 [00:48<33:25:22, 24.07s/it]


KeyboardInterrupt: 